In [1]:
import os

# Your desired directory path
desired_path = "c:/Projects/taxi_demand_predictory/"

# Change the current working directory to the specified path
os.chdir(desired_path)

# Verify by printing the current working directory
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  c:\Projects\taxi_demand_predictory


In [2]:
import src.config as cfg

In [3]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data from the last 28 days, to add redundancy to the data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2023-09-18 03:00:00')


In [4]:
from src.data import load_raw_data

# simulate a call to a data source/warehouse
def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)"""
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [5]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2023-08-21 03:00:00'), to_date_=Timestamp('2022-09-19 03:00:00')
File 2022-08 was already in local storage
File 2022-09 was already in local storage


In [6]:
rides.sort_values(by=['pickup_datetime'])

,pickup_datetime,pickup_location_id
2075059,2023-08-21 03:00:04,162
2075229,2023-08-21 03:00:07,230
2074517,2023-08-21 03:00:12,48
2075214,2023-08-21 03:00:23,230
2075158,2023-08-21 03:00:31,186
...,...,...
1773473,2023-09-18 02:59:07,132
1773430,2023-09-18 02:59:11,141
1773174,2023-09-18 02:59:13,249
1773149,2023-09-18 02:59:45,163


In [10]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:00<00:00, 450.09it/s]


In [14]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=cfg.HOPSWORKS_PROJECT_NAME,
    api_key_value=cfg.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=cfg.FEATURE_GROUP_NAME,
    version=cfg.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/104604
Connected. Call `.close()` to terminate connection gracefully.


In [15]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 0.00% |          | Rows 0/178080 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/104604/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x162dc2789a0>, None)